<a href="https://colab.research.google.com/github/Liza-IITP/Linear-Logistic/blob/main/Bayesian_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Define Bayesian Networks
import networkx as nx
from itertools import product
import matplotlib.pyplot as plt
class BayesNet(nx.DiGraph):
    def __mod__(self, node): return self.nodes[node]['V']
    def __mul__(self, other): n, v = other; self.add_node(n, V=tuple(v), CPT=None); return self
    def __truediv__(self, other):
        for o in other:  self.remove_node(o); return self
    def __add__(self, other): self.add_edge(*other); return self
    def __sub__(self, other): self.remove_edge(*other); return self
    def __call__(self, node, evi=None): return self.nodes[node]['CPT'] if evi is None else self.nodes[node]['CPT'][evi]
    def __invert__(self,):
        for n in self.nodes:
            pdn = self.predecessors(n)
            vals = (self)%(n) # set of possible vals
            pvals=[(self)%(p) for p in pdn] # list of sets
            self.nodes[n]['CPT'] = {combo:{k:0.0 for k in vals} for combo in product(*pvals)}
            self.nodes[n]['P'] = tuple(pdn)
        return self
    def check(self, delta=1e-15):
        for n in self.nodes:
            print(f'Node [{n}]')
            cpt = self.nodes[n]['CPT']
            for k, v in cpt.items():
                sumprobs = sum(list(v.values()))
                print(f'\tEvi [{k}], probs={v}, {sumprobs=}, {"✅" if abs(1 - sumprobs) < delta else "⛔"}')
    def plot(self, pos=None, figsize=(5,5), node_color='white', edge_color='black', node_size=500, font_size=10, arrow_size=10, seed=None):
        if not pos: pos = nx.spring_layout(self, seed=seed)
        plt.figure(figsize=figsize)
        nx.draw(self, pos, with_labels=True, node_size=node_size, node_color=node_color, font_size=font_size, font_weight="bold", arrowsize=arrow_size, edgecolors=edge_color,)
        plt.title(f"Nodes={len(self.nodes)} Edges={len(self.edges)}")
        plt.axis("off")
        plt.show()


In [4]:

BN = BayesNet()
BN *= ('E', ('E0', 'E1'))
BN *= ('B', ('B0', 'B1'))
BN *= ('A', ('A0', 'A1'))
BN *= ('R', ('R0', 'R1'))
BN *= ('W', ('W0', 'W1'))
BN *= ('G', ('G0', 'G1'))
BN += ('E', 'R')
BN += ('E', 'A')
BN += ('B', 'A')
BN += ('A', 'W')
BN += ('A', 'G')
~BN
BN('A', ('E0', 'B0'))['A1'] = 0.01
BN('A', ('E0', 'B0'))['A0'] = 0.99
BN('A', ('E1', 'B0'))['A1'] = 0.2
BN('A', ('E1', 'B0'))['A0'] = 0.8
BN('A', ('E0', 'B1'))['A1'] = 0.95
BN('A', ('E0', 'B1'))['A0'] = 0.05
BN('A', ('E1', 'B1'))['A1'] = 0.96
BN('A', ('E1', 'B1'))['A0'] = 0.04
BN('B',())['B1']=0.0001
BN('B',())['B0']=0.9999
BN('E',())['E1']=0.0003
BN('E',())['E0']=0.9997
BN('R', ('E0',))['R1'] = 0.0002
BN('R', ('E1',))['R1'] = 0.9
BN('R', ('E1',))['R0'] = 0.1
BN('R', ('E0',))['R0'] = 0.9998
BN('W', ('A0',))['W1'] = 0.4
BN('W', ('A0',))['W0'] = 0.6
BN('W', ('A1',))['W1'] = 0.8
BN('W', ('A1',))['W0'] = 0.2
BN('G', ('A0',))['G1'] = 0.04
BN('G', ('A0',))['G0'] = 0.96
BN('G', ('A1',))['G1'] = 0.4
BN('G', ('A1',))['G0'] = 0.6
BN.check()

# q1-A,B',E',W,G,R'

def joint_alarm_event(BN):
    p_B0 = BN('B', ())['B0']
    p_E0 = BN('E', ())['E0']
    p_A1 = BN('A', ('E0','B0'))['A1']
    p_R0 = BN('R', ('E0',))['R0']
    p_W1 = BN('W', ('A1',))['W1']
    p_G1 = BN('G', ('A1',))['G1']
    return p_B0 * p_E0 * p_A1 * p_R0 * p_W1 * p_G1

print("Q.1-> ANS:  Probability of all events intersecting:")
print(joint_alarm_event(BN))

# q2-A',B',E',W',G',R'

def joint_alarm_event2(BN):
    p_B0 = BN('B', ())['B0']
    p_E0 = BN('E', ())['E0']
    p_A0 = BN('A', ('E0','B0'))['A0']
    p_R0 = BN('R', ('E0',))['R0']
    p_W0 = BN('W', ('A0',))['W0']
    p_G0 = BN('G', ('A0',))['G0']
    return p_B0 * p_E0 * p_A0 * p_R0 * p_W0 * p_G0

print("Q.2-> ANS :  Probability of all events intersecting:")
print(joint_alarm_event2(BN))



Node [E]
	Evi [()], probs={'E0': 0.9997, 'E1': 0.0003}, sumprobs=1.0, ✅
Node [B]
	Evi [()], probs={'B0': 0.9999, 'B1': 0.0001}, sumprobs=1.0, ✅
Node [A]
	Evi [('E0', 'B0')], probs={'A0': 0.99, 'A1': 0.01}, sumprobs=1.0, ✅
	Evi [('E0', 'B1')], probs={'A0': 0.05, 'A1': 0.95}, sumprobs=1.0, ✅
	Evi [('E1', 'B0')], probs={'A0': 0.8, 'A1': 0.2}, sumprobs=1.0, ✅
	Evi [('E1', 'B1')], probs={'A0': 0.04, 'A1': 0.96}, sumprobs=1.0, ✅
Node [R]
	Evi [('E0',)], probs={'R0': 0.9998, 'R1': 0.0002}, sumprobs=1.0, ✅
	Evi [('E1',)], probs={'R0': 0.1, 'R1': 0.9}, sumprobs=1.0, ✅
Node [W]
	Evi [('A0',)], probs={'W0': 0.6, 'W1': 0.4}, sumprobs=1.0, ✅
	Evi [('A1',)], probs={'W0': 0.2, 'W1': 0.8}, sumprobs=1.0, ✅
Node [G]
	Evi [('A0',)], probs={'G0': 0.96, 'G1': 0.04}, sumprobs=1.0, ✅
	Evi [('A1',)], probs={'G0': 0.6, 'G1': 0.4}, sumprobs=1.0, ✅
Q.1-> ANS:  Probability of all events intersecting:
0.0031980803519808
Q.2-> ANS :  Probability of all events intersecting:
0.5698979187229786


In [9]:
import math
from collections import Counter
import pandas as pd

V = ['I','loved','the','movie','hated','a','great','poor','acting','good']

docs = [
    "I loved the movie",                # +
    "I hated the movie",                # -
    "a great movie good movie",         # +
    "poor acting",                      # -
    "great acting a good movie"         # +
]

labels = ['+','-','+','-','+']  # corresponding labels
alpha = 1.0  # Laplace smoothing

# Priors
N = len(labels)
class_counts = Counter(labels)
priors = {c: class_counts[c]/N for c in class_counts}

# Counts per class
word_counts = {c: Counter() for c in class_counts}
total_words_in_class = {c: 0 for c in class_counts}
for doc, lab in zip(docs, labels):
    for w in doc.split():
        word_counts[lab][w] += 1
        total_words_in_class[lab] += 1

V_size = len(V)
likelihoods = {c: {} for c in class_counts}
for c in class_counts:
    denom = total_words_in_class[c] + alpha * V_size
    for w in V:
        count = word_counts[c][w]
        likelihoods[c][w] = (count + alpha) / denom

# Sentence
sentence = "I hated the poor acting"
s_words = sentence.split()

# Log-posteriors
log_post = {}
for c in class_counts:
    logp = math.log(priors[c])
    for w in s_words:
        logp += math.log(likelihoods[c].get(w, alpha / (total_words_in_class[c] + alpha * V_size)))
    log_post[c] = logp
max_log = max(log_post.values())
exp_post = {c: math.exp(log_post[c] - max_log) for c in log_post}
sum_exp = sum(exp_post.values())
posteriors = {c: exp_post[c] / sum_exp for c in exp_post}
print("Priors:", priors)
print("Total words per class:", total_words_in_class)
print("Likelihoods (P(word|class)):\n")
print(pd.DataFrame(likelihoods))
print("\nLog-scores:", log_post)
print("\nPosteriors:", posteriors)
print("\nUnnormalized scores (as in the slide):")
for c in class_counts:
    score = priors[c]
    for w in s_words:
        score *= likelihoods[c][w]
    print(f" P({c}) * ∏ P(word|{c}) = {score:.6e}")


Priors: {'+': 0.6, '-': 0.4}
Total words per class: {'+': 14, '-': 6}
Likelihoods (P(word|class)):

               +       -
I       0.083333  0.1250
loved   0.083333  0.0625
the     0.083333  0.1250
movie   0.208333  0.1250
hated   0.041667  0.1250
a       0.125000  0.0625
great   0.125000  0.0625
poor    0.041667  0.1250
acting  0.083333  0.1250
good    0.125000  0.0625

Log-scores: {'+': -14.321653233825883, '-': -11.313498440273335}

Posteriors: {'+': 0.0470588235294118, '-': 0.9529411764705883}

Unnormalized scores (as in the slide):
 P(+) * ∏ P(word|+) = 6.028164e-07
 P(-) * ∏ P(word|-) = 1.220703e-05
